In [1]:
import pandas as pd

from rdkit import Chem
from ligand_graph_features import *
from torch_geometric.data import DataLoader

from model_Yang import GNN
from torch_geometric.utils import to_dense_batch

In [2]:
dataset ='tier12'

contextpred_config={
                 'num_layer': 5,
                 'emb_dim': 300,
                 'JK': 'last',
                 'drop_ratio': 0.5,
                 'gnn_type': 'gin'
             }

In [3]:
ligandEmbedding = GNN(num_layer=contextpred_config['num_layer'],
                       emb_dim=contextpred_config['emb_dim'],
                       JK=contextpred_config['JK'],
                       drop_ratio=contextpred_config['drop_ratio'],
                       gnn_type=contextpred_config['gnn_type'])

In [4]:
setting='test'

In [5]:
train = pd.read_csv(f'data/{setting}/{dataset}-all.csv')

In [6]:
train.shape

(38533, 3)

In [7]:
usable =[]
problematic = []
i=0
for entry in list(train.values):
    s = entry[0]
    try:
        mol = Chem.MolFromSmiles(s)
        graph = mol_to_graph_data_obj_simple(mol)
        chem_graph_list =  [graph]
        chem_graphs_loader = DataLoader(chem_graph_list, batch_size=1,
                                shuffle=False)
        for batch in chem_graphs_loader:
            chem_graphs = batch
        node_representation = ligandEmbedding(chem_graphs.x, chem_graphs.edge_index,
                                     chem_graphs.edge_attr)
        batch_chem_graphs_repr_masked, mask_graph = to_dense_batch(node_representation,
                                                        chem_graphs.batch)
        usable.append(entry)
        i+=1
    except:
        print(i)
        problematic.append(i)
        i+=1
        

766
2384
7203
8144
8885
8955
15460
17219
21007
22655
22809
23083
34013
34884
35754


RDKit WARNING: [05:51:33] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:33] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [05:51:34] WARNING: not removing hydrogen atom without neighbors


In [15]:
problem_ends = set()
for i in problematic:
    e = train['endpoint'][i]
    problem_ends.add(e)
    print(e)

log_s
herg_inhibitor
cyp1a2_inhibition
cyp1a2_inhibition
cyp1a2_inhibition
cyp1a2_inhibition
cyp2d6_inhibition
cyp2d6_inhibition
cyp2c9_inhibition
cyp2c9_inhibition
cyp2c9_inhibition
cyp2c9_inhibition
cyp2c19_inhibition
cyp2c19_inhibition
cyp2c19_inhibition


In [18]:
np.save('data/problem_ends_for_single_task.npy',np.array(problem_ends))

In [11]:
len(usable)-train.shape[0]

-15

In [12]:
df_usable = pd.DataFrame(usable,columns = train.columns)

In [13]:
df_usable.head()

,similes,y,endpoint
0,CC(=O)OC1C2C(CC(C)C3C=CC(=O)C31C)OC(=O)C2C,1,log_s
1,C=C(Cl)C(N)=O,1,log_s
2,CC1(C)C(O)C(C)(C)C1O,1,log_s
3,O=[N+]([O-])c1ccn[nH]1,1,log_s
4,C=CCCC(=O)C1=CCCC(C)(C)C1,1,log_s


In [14]:
df_usable.to_csv(f'data/{setting}/{dataset}-all.csv',index=None)